In [1]:
#!pip install numpy
#!pip install prettytable

In [2]:
import random

import numpy as np

from prettytable import PrettyTable

In [3]:
class Layer:
    def __init__(self, input_size, output_size, activation_func):
        self.activate, self.derivative = activation_func()
        self.weights = np.random.uniform(-1.0,1.0,size=(input_size, output_size))
        self.biases = np.zeros(output_size)        
    def __call__(self):
        return {
            'weights':self.weights, 
            'biases':self.biases
                }

In [4]:
class Activation:
    @staticmethod
    def SIGMOID():
        def activation(x):
            return 1 / (1 + np.exp(-x))
        def derivative(x):
            return x * (1 - x)
        return activation, derivative
    
    @staticmethod
    def TANH():
        def activation(x):
            return np.tanh(x)
        def derivative(x):
            return 1 - x ** 2
        return activation, derivative


In [5]:
class NN:
    def __init__(self, layers):
        self.layers = layers
        self.params = self.__params()
    
    def __params(self):
        params = {}
        for i, layer in enumerate(layers):
            params[f'layer{i}'] = {"weights":layer.weights,
                                   "biases":layer.biases}
        return params
    def __forward(self, x):
        for layer in self.layers:
            z = np.dot(x, layer.weights) + layer.biases
            x = layer.activate(z)
            layer.output = np.array([x])
        return x
    
    def __backward(self, x, y, lr):
        error = -(y - self.layers[-1].output)
        for layer in reversed(self.layers):
            delta = error * layer.derivative(layer.output)
            prev_output = np.array([x]) if layer == self.layers[0] else self.layers[self.layers.index(layer) - 1].output
            
            layer.weights -= lr * np.dot(prev_output.T, delta)
            layer.biases -= lr * np.sum(delta, axis=0)
            error = np.dot(delta, layer.weights.T)

    def train(self, input_data, output_data, learning_rate=0.01):
        index_list = list(range(input_data.shape[0]))
        all_correct = False
        while not all_correct:
            all_correct = True
            random.shuffle(index_list)
            for i in index_list: 
                self.__forward(input_data[i])
                self.__backward(input_data[i], output_data[i][0], learning_rate)
            for i in range(input_data.shape[0]): 
                p_out = self.__forward(input_data[i])
                all_correct = False if(((output_data[i] < 0.5) and (p_out >= 0.5)) or ((output_data[i] >= 0.5) and (p_out < 0.5))) else all_correct  
    
    
    def __call__(self, input_data):
        output = []
        for i in range(input_data.shape[0]):
            output.append(1 if self.__forward(input_data[i]) >= 0.5 else 0)
        return  output
    

In [6]:
LEARNING_RATE = 0.1
#~~Тренировочные данные~~#
train_input = np.array([[0, 0, 0],
              [0, 0, 1],
              [0, 1, 0],
              [0, 1, 1],
              [1, 0, 0],
              [1, 0, 1],
              [1, 1, 0],
              [1, 1, 1]])
train_output = np.array([[0,0,0,0,0,0,1,0]]).T

In [7]:
#~~Структура нейросети~~#
layers = (
    Layer(train_input.shape[1], 4, Activation.TANH),
    Layer(4, 4, Activation.TANH),
    Layer(4, 4, Activation.TANH),
    Layer(4, 1, Activation.SIGMOID),
)

In [8]:
model = NN(layers) #создание объекта нейросети
print(model.params) #вывод параметров

{'layer0': {'weights': array([[-0.12569597,  0.53664828, -0.07118578, -0.15411523],
       [-0.80348131, -0.36376164, -0.49611482, -0.48716103],
       [ 0.82754344,  0.75038322,  0.44968188,  0.65198094]]), 'biases': array([0., 0., 0., 0.])}, 'layer1': {'weights': array([[ 0.28538318, -0.40570441,  0.92044151,  0.95172123],
       [-0.18987646, -0.8123381 ,  0.41979125,  0.7833092 ],
       [-0.47337865, -0.53352738, -0.81193015,  0.97712457],
       [-0.2395171 ,  0.87379524, -0.99647523,  0.32474638]]), 'biases': array([0., 0., 0., 0.])}, 'layer2': {'weights': array([[ 0.55075226, -0.67010772,  0.97542937, -0.9533303 ],
       [ 0.72753401,  0.805766  , -0.51820003, -0.38344259],
       [ 0.74956863, -0.44144785,  0.57329522, -0.80201082],
       [-0.50005348, -0.77355954,  0.98072434, -0.07465803]]), 'biases': array([0., 0., 0., 0.])}, 'layer3': {'weights': array([[-0.39683331],
       [ 0.81750564],
       [-0.6078506 ],
       [-0.46523121]]), 'biases': array([0.])}}


In [9]:
#~~Обучение нейросети~~#
model.train(train_input, train_output, learning_rate=LEARNING_RATE) 

In [10]:
print(model.params) #вывод параметров

{'layer0': {'weights': array([[-0.27893624,  0.30033068, -0.08638879,  0.0091776 ],
       [-0.74061045, -0.38163049, -0.497881  , -0.54111852],
       [ 0.89391811,  0.79047645,  0.4753022 ,  0.60303396]]), 'biases': array([ 0.26497781,  0.1195622 ,  0.06167893, -0.16768157])}, 'layer1': {'weights': array([[ 0.3448959 , -0.4653392 ,  0.9422645 ,  0.99664469],
       [-0.25293352, -0.74433882,  0.39036861,  0.77892137],
       [-0.43976173, -0.56367238, -0.80269189,  1.0011556 ],
       [-0.20461946,  0.8451374 , -0.99024692,  0.34560333]]), 'biases': array([ 0.05798559, -0.0999723 ,  0.0661497 ,  0.08560772])}, 'layer2': {'weights': array([[ 0.55025211, -0.67598381,  0.97982865, -0.95220805],
       [ 0.69224953,  0.78926409, -0.50803512, -0.42667338],
       [ 0.77625252, -0.43529759,  0.57054859, -0.76599317],
       [-0.44457191, -0.84298005,  1.03490592,  0.0034418 ]]), 'biases': array([ 0.10573949, -0.1116673 ,  0.08951186,  0.1660092 ])}, 'layer3': {'weights': array([[-0.3306658

In [11]:
pt = PrettyTable(['x1','x2','x3','F'])
for i in range(train_input.shape[0]):
    pt.add_row([train_input[i][0], train_input[i][1],train_input[i][2], model(train_input)[i]])
print(pt)


+----+----+----+---+
| x1 | x2 | x3 | F |
+----+----+----+---+
| 0  | 0  | 0  | 0 |
| 0  | 0  | 1  | 0 |
| 0  | 1  | 0  | 0 |
| 0  | 1  | 1  | 0 |
| 1  | 0  | 0  | 0 |
| 1  | 0  | 1  | 0 |
| 1  | 1  | 0  | 1 |
| 1  | 1  | 1  | 0 |
+----+----+----+---+
